In [1]:
#!pip install python-sat

In [11]:
from pysat.formula import CNF
from google.colab import files

In [3]:
cnf = CNF()

#Generate CNF (file .cnf) automatically using pysat

Predicate:

1. Revealed(i, j): Represents that the tile at position (i, j) is revealed. (opposite is unrevealed) **(Denoted as variable x1)**
2. Trap(x, y): Represents that the tile at position (i, j) is a trap. (opposite is gem) **(Denoted as variable x2)**
3. Surrounding(i1, j1, i2, j2): Represents that the tile at position (i2, j2) is a neighbor of the tile at position (i1, j1). **(Denoted as variable x3)**
4. Begin(i, j, k): Represents that tile at position (i, j) is a beginning tile that has k unrevealed trap surrounding it. **(Denoted as variable x4)**

Function:

1. countUnrevealedTiles(i, j): Return the number of surrounding unrevealed tiles of the tile at position (i, j)
2. countRevealedTrapTiles(i, j): Return the number of trapped revealed surrounding tiles of the tile at position (i, j)

Addition:

1. (countUnrevealedTiles(i1, j1) + countRevealedTrapTiles(i1, j1) = k) **(Denoted as variable x5)**
2. (k = 0) **(Denoted as variable x6)**
3. (countRevealedTrapTiles(i1, j1) = k) **(Denoted as variable x7)**



##Constain 1: Each beginning tile has a number represents the number of traps (defined as 'k') surrounding it
1. FOL
    - ∀i, j, ∃k Begin(i, j, k)
2. CNF
    - Begin(i, j, k)
3. Denote
    - x4


In [4]:
cnf.append([4])
print(cnf.clauses)

[[4]]


##Constraint 2: If number of unrevealed tiles surrounding plus number of trapped tiles revealed is equal to k then all unrevealed tiles surrounding are traps and are revealed.
1. FOL:
    - ∀i1, j1, ∃k (Begin(i1, j,1 k) ∧ (countUnrevealedTiles(i1, j1) + countRevealedTrapTiles(i1, j1) = k)) ⇒ (∀i2, j2 (Surrounding(i1, j1, i2, j2) ∧ ~Revealed(i2, j2)) ⇒ (Revealed(i2, j2) ∧ Trap(i2, j2)))
2. CNF:
    - ~Begin(i1, j,1 k) V ~(countUnrevealedTiles(i1, j1) + countRevealedTrapTiles(i1, j1) = k) V ~Surrounding(i1, j1, i2, j2) V Revealed(i2, j2)
    - ~Begin(i1, j,1 k) V ~(countUnrevealedTiles(i1, j1) + countRevealedTrapTiles(i1, j1) = k) V ~Surrounding(i1, j1, i2, j2) V Revealed(i2, j2) V Trap(i2, j2)
3. Denote:
    - -x4 V -x5 V -x3 V x1
    - -x4 V -x5 V -x3 V x1 V x2

In [5]:
cnf.append([-4, -5, -3, 1])
cnf.append([-4, -5, -3, 1, 2])
print(cnf.clauses)

[[4], [-4, -5, -3, 1], [-4, -5, -3, 1, 2]]


##Constaint 3: If k is equal to 0 then all surrounding tiles are not trap (which mean it is revealed as gem if it is an unrevealed tile)

1. FOL
    - ∀i1, j1, ∃k (Begin(i1, j1, k) ∧ (k = 0) ∧ (∀i2, j2 \~Revealed(i2, j2) ∧ Surrounding(i1, j1, i2, j2))) ⇒ (\~Trap(i2, j2) ∧ Revealed(i2, j2))
2. CNF
    - ~Begin(i1, j1, k) V ~(k = 0) V Revealed(i2, j2) V ~Surrounding(i1, j1, i2, j2) V ~Trap(i2, j2)
    - ~Begin(i1, j1, k) V ~(k = 0) V Revealed(i2, j2) V ~Surrounding(i1, j1, i2, j2)
3. Detone:
    - -x4 V -x6 V x1 V -x3 V -x2
    - -x4 V -x6 V x1 V -x3

In [6]:
cnf.append([-4, -6, 1, -3, -2])
cnf.append([-4, -6, 1, -3])
print(cnf.clauses)

[[4], [-4, -5, -3, 1], [-4, -5, -3, 1, 2], [-4, -6, 1, -3, -2], [-4, -6, 1, -3]]


##Constrain 4: If there are enough k surrounding traps being revealed then all remaining surrounding unrevealed tiles are gems.
1. FOL:
    - ∀i1, j1, ∃k (Begin(i1, j1, k) ∧ (countRevealedTrapTiles(i1, j1) = k)) ⇒ (∀i2, j2 (Surrounding(i1, j1, i2, j2) ∧ \~Revealed(i2, j2)) ⇒ (\~Trap(i2, j2) ∧ Revealed(i2, j2)))
2. CNF:
    - ~Begin(i1, j1, k) V ~(countRevealedTrapTiles(i1, j1) = k) V ~Surrounding(i1, j1, i2, j2) V Revealed(i2, j2) V ~Trap(i2, j2)
    - ~Begin(i1, j1, k) V ~(countRevealedTrapTiles(i1, j1) = k) V ~Surrounding(i1, j1, i2, j2) V Revealed(i2, j2)
3. Denote:
    - -x4 V -x7 V -x3 V x1 V -x2
    - -x4 V -x7 V -x3 V x1

In [7]:
cnf.append([-4, -7, -3, 1, -2])
cnf.append([-4, -7, -3, 1])
print(cnf.clauses)

[[4], [-4, -5, -3, 1], [-4, -5, -3, 1, 2], [-4, -6, 1, -3, -2], [-4, -6, 1, -3], [-4, -7, -3, 1, -2], [-4, -7, -3, 1]]


##Constrain 5: For every trapped tile, there exists at least one beginning tile in its surrounding.
1. FOL
    - ∀i1, j1 Trap(i1, j1) ⇒ ∃i2, j2, k Begin(i2, j2, k) ∧ Surrounding(i1, j1, i2, j2)
2. CNF
    - ~Trap(i1, j1) V Begin(i2, j2, k)
    - ~Trap(i1, j1) V Surrounding(i1, j1, i2, j2))
3. Denote
    - -x2 V x4
    - -x2 V x3

In [8]:
cnf.append([-2, 4])
cnf.append([-2, 3])
print(cnf.clauses)

[[4], [-4, -5, -3, 1], [-4, -5, -3, 1, 2], [-4, -6, 1, -3, -2], [-4, -6, 1, -3], [-4, -7, -3, 1, -2], [-4, -7, -3, 1], [-2, 4], [-2, 3]]


##Constraint 6: Each unrevealed tile can either be a trap or gem
1. FOL:
    - ∀i, j ~Revealed(i, j) ⇒ Trap(i, j) V ~Trap(i, j)
2. CNF:
    - Revealed(i, j) V Trap(i, j) V ~Trap(i, j)
3. Denote:
    - x1 V x2 V -x2

In [9]:
cnf.append([1, 2, -2])
print(cnf.clauses)

[[4], [-4, -5, -3, 1], [-4, -5, -3, 1, 2], [-4, -6, 1, -3, -2], [-4, -6, 1, -3], [-4, -7, -3, 1, -2], [-4, -7, -3, 1], [-2, 4], [-2, 3], [1, 2, -2]]


##Write to file

In [10]:
cnf.to_file('GameRules.cnf')

##Download file

In [12]:
#files.download("GameRules.cnf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>